In [9]:
import numpy as np
import pandas as pd
import re

#Función para eliminar los símbolos, emoticones y links
def clean_text(text):
    # Eliminar URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    
    # Definir un patrón de regex para eliminar emoticones, símbolos y caracteres no deseados
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        u"\U0001F700-\U0001F77F"  # Alchemical Symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F1E0-\U0001F1FF"  # Flags
        "]+", flags=re.UNICODE
    )
    
    # Eliminar emoticones, símbolos y caracteres especiales
    text = emoji_pattern.sub(r'', text)
    
    # Mantener solo letras y espacios (eliminar números, puntuación, símbolos matemáticos, etc.)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar espacios extra
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Cargar la base de datos
tweets_file = "tweets_Jun20.csv"
df_tweets = pd.read_csv(tweets_file)

# Eliminar tweets que no tienen texto
df_tweets_clean = df_tweets.dropna(subset=['Texto'])

# Aplicar la función de limpieza a la columna de texto
df_tweets_clean['Texto'] = df_tweets_clean['Texto'].apply(clean_text)

# Guardar
df_tweets_clean.to_csv('tweets_Jun20_cleaned.csv', index=False)

/var/folders/8b/d34q145x44x2xf_06_gchyfw0000gq/T/ipykernel_10897/953605553.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_clean['Texto'] = df_tweets_clean['Texto'].apply(clean_text)


In [12]:
df_tweets_clean['Texto']

1       cuidadodelagua sin suministro de agua en la co...
2       serviciodeelectricidad sin servicio en a aveni...
3       banquetaobstruida por vehculos estacionados en...
4       transportepblico rutas que suspenden labores a...
5       atencin sigue las recomendaciones que aqu te c...
                              ...                        
2245    una de las mejores propuestas ciudadanas que h...
2246    pues marthaherreranl ha hecho ms cosas por nue...
2247    otra historia registrada en mi querida av pase...
2248                                          sannicolsnl
2249                            y lateral tipicodeviernes
Name: Texto, Length: 2229, dtype: object